In [2]:
import konlpy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

In [2]:
import pickle
with open('train_moong.pickle', 'rb') as f:
    train_x = pickle.load(f) # 단 한줄씩 읽어옴
with open('train_label_moong.pickle', 'rb') as f:
    train_y  = pickle.load(f) # 단 한줄씩 읽어옴

In [4]:
# 긍정 / 혹은 부정 문장리스트
def wordCount(data, word):
    cnt = 0
    for sentence in data:
        if word in sentence:
            cnt += 1
    return cnt

# data : 긍/ 부정 문서 리스트 
# data[0] : 부정 문서 싹다.
# data[1] : 긍정 문서 싹다. 
# w : 단어 
# c : 긍정, 부정
# return 카이제곱 통계량
def getKai(datas, word, isPositive):
    A = wordCount(datas[isPositive], word)
    B = wordCount(datas[(isPositive+1)%2], w)
    C = len(datas[isPositive]) - A
    D = len(datas[(isPositive+1)%2]) - B
    # 분모가 0이면 0을 리턴함.
    if ((A+B)*(A+C)*(B+D)*(C+D)) == 0:
        return 0
    return ((A+B+C+D)*((A*D-B*C)*(A*D-B*C))) / ((A+B)*(A+C)*(B+D)*(C+D))

146182

In [ ]:
train_x = list(train_x)
train_y = list(train_y)

In [ ]:
# words 는 현이님이 주신 피클의 모든 단어들
words = []
for s in train_x:
    for w in s:
        words.append(w)
# 중복 제거
words = list(set(words))
# 긍/부정 문서로 나눈다.
data, dataP, dataN = [], [], []
for i in range(len(train_x)):
    if(train_y[i] == 1):
        dataP.append(train_x[i])
    else:
        dataN.append(train_x[i])
data =[dataN, dataP]
print(data)

In [ ]:
kai_dict = []
for w in tqdm(words):
    kai_dict.append(getKai(data, w, 0))
kaiRate = pd.DataFrame()
kaiRate['words'] = words
kaiRate.index = words
kaiRate['word_kai'] = kai_dict
# kaiRate은 카이제곱 정렬 상위 단어 
kaiRate = kaiRate.sort_values(by=['word_kai'],ascending=False)

In [ ]:
const_kai   = 1000   #구할 카이제곱상위 단어의 개수
realkaiRate = kaiRate[:const_kai]
realkaiRate = list(realkaiRate["words"])

In [ ]:
v1 = CountVectorizer()
v2 = CountVectorizer()
# 기본단어
cntVectorizer_words = v1.fit(words)
# 카이제곱 상위 단어
cntVectorizer_kai = v2.fit(realkaiRate)

In [ ]:
from sklearn.model_selection import train_test_split

with open('X_sample.pickle', 'rb') as f:
    sample_x = pickle.load(f) # 단 한줄씩 읽어옴
    
with open('y_sample.pickle', 'rb') as f:
    sample_y  = pickle.load(f) # 단 한줄씩 읽어옴

In [ ]:
# 바로 sample_x를 countVectorizer에 박으려고했으나 리스트라서 안된다고함.
# 스트링으로 바꿔줆
# sample_x = [['안녕', '나는', '문근'], ['너는', '안녕'], ....]
# xx = ["안녕 나는 문근", "너는 안녕", "ㅎㅇ ㅎㅇ" .... ]
xx = []
for s in sample_x:
    ss = ""
    for w in s:
        ss += (w + " ")
    xx.append(ss)
sample_x = xx

In [ ]:
# 희소행렬 두개를 합쳐벌임
def merge_sparse(a, b):
    return sparse.csr_matrix(np.hstack((a.toarray(), b.toarray())))

X_train,X_test,y_train,y_test=train_test_split(sample_x, sample_y,random_state=2020)
# 기본단어 feature
w = cntVectorizer_words.transform(X_train)
# 카이제곱 상위 단어 feature
k = cntVectorizer_kai.transform(X_train)
X_train = merge_sparse(w, k) 


# 기본단어 features
w = cntVectorizer_words.transform(X_test)
# 카이제곱 상위 단어 features
k = cntVectorizer_kai.transform(X_test)
X_test = merge_sparse(w, k)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

classifier = MultinomialNB()

classifier.fit(X_train, y_train)

pred = classifier.predict(X_test).tolist()

print('Accuracy1: %.10f' % accuracy_score(y_test, pred))